In [1]:
import pandas as pd
import re
from langdetect import detect,detect_langs,DetectorFactory
import emoji
from pprint import pprint

# Ensure consistent language detection results
DetectorFactory.seed = 0

In [2]:
file_path = 'data/final_video_data.csv'
videos_df = pd.read_csv(file_path)
videos_df

,c_channel_id,v_video_id,v_date_publishedAt,v_time_publishedAt,c_channel_title,c_channel_upload_playlist,c_channel_subCount,c_channel_videoCount,c_channel_viewCount,v_category_id,...,v_comment_count,v_likes,v_favorites,v_views,v_definition,v_duration,v_caption,v_age_restricted,v_privacy_status,v_made_for_kids
0,UC295-Dw_tDNtZXFeAPAW6Aw,juYonFfdxJQ,2024-09-10,13:00:53,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,23,491,0,62211,hd,1:00:39,False,Not Restricted,public,False
1,UC295-Dw_tDNtZXFeAPAW6Aw,oGga7hZypCs,2024-09-02,11:00:48,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,28,442,0,76602,hd,2:03:35,False,Not Restricted,public,False
2,UC295-Dw_tDNtZXFeAPAW6Aw,pkzIiXkRxB0,2024-08-30,11:00:54,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,21,1165,0,341133,hd,2:01:48,False,Not Restricted,public,False
3,UC295-Dw_tDNtZXFeAPAW6Aw,EGzGak2IyrA,2024-08-30,13:00:13,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,27,344,0,65712,hd,1:00:39,False,Not Restricted,public,False
4,UC295-Dw_tDNtZXFeAPAW6Aw,TUS8vZaHNds,2024-08-31,11:00:26,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,43,1374,0,222425,hd,1:00:04,False,Not Restricted,public,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45382,UCx2grh-o99YnDXcy0Pf7clg,vSDsYrWQUio,2024-12-05,04:53:38,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,0,0,1,hd,0:04:17,False,Not Restricted,public,False
45383,UCx2grh-o99YnDXcy0Pf7clg,3i2Gxbuf8v4,2024-12-09,11:20:11,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,2,0,3,hd,0:16:12,False,Not Restricted,public,False
45384,UCx2grh-o99YnDXcy0Pf7clg,X-Rj8IGrACo,2024-12-15,21:43:42,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,1,0,3,hd,0:01:23,False,Not Restricted,public,False
45385,UCx2grh-o99YnDXcy0Pf7clg,aJjO1LGtG6U,2024-11-30,14:15:58,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,1,0,3,hd,0:09:26,False,Not Restricted,public,False


In [3]:
def is_emoji(char):
    """Check if a character is an emoji."""
    return char in emoji.EMOJI_DATA

In [4]:
def remove_emojis(text):
    """Remove emojis from a string."""
    return ''.join(char for char in text if not is_emoji(char))

In [5]:
def clean_text(text):
    """Remove special characters, URLs, and keep only meaningful text."""
    text = remove_emojis(text)  # Remove emojis
    text = re.sub(r'[\u200b\u2122\u00AE]', '', text)  # Remove zero-width spaces, trademark symbols
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only English letters and spaces
    return text.strip()

In [6]:
def is_mostly_english(text):
    """Check if the majority of the text is English (at least 70% English)."""
    if not text or pd.isna(text):  # Handle empty or NaN values
        return True  # Treat empty as English to avoid unnecessary row deletion
    try:
        cleaned_text = clean_text(text)
        lang_probs = detect_langs(cleaned_text)
        for lang in lang_probs:
            if lang.lang == 'en' and lang.prob > 0.7:  # At least 70% English
                return True
    except:
        return False  # If detection fails, assume non-English
    return False

In [7]:
# c_channel_title, v_title, v_description, v_tags
# Apply the function to filter DataFrame
cleaned_videos_df = videos_df[
    videos_df[['v_title']]
    .applymap(is_mostly_english)
    .all(axis=1)  # Keep only rows where all columns are mostly English
]

# Display cleaned DataFrame
cleaned_videos_df

C:\Users\papad\AppData\Local\Temp\ipykernel_54616\1226015698.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(is_mostly_english)


,c_channel_id,v_video_id,v_date_publishedAt,v_time_publishedAt,c_channel_title,c_channel_upload_playlist,c_channel_subCount,c_channel_videoCount,c_channel_viewCount,v_category_id,...,v_comment_count,v_likes,v_favorites,v_views,v_definition,v_duration,v_caption,v_age_restricted,v_privacy_status,v_made_for_kids
1,UC295-Dw_tDNtZXFeAPAW6Aw,oGga7hZypCs,2024-09-02,11:00:48,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,28,442,0,76602,hd,2:03:35,False,Not Restricted,public,False
2,UC295-Dw_tDNtZXFeAPAW6Aw,pkzIiXkRxB0,2024-08-30,11:00:54,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,21,1165,0,341133,hd,2:01:48,False,Not Restricted,public,False
4,UC295-Dw_tDNtZXFeAPAW6Aw,TUS8vZaHNds,2024-08-31,11:00:26,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,43,1374,0,222425,hd,1:00:04,False,Not Restricted,public,False
5,UC295-Dw_tDNtZXFeAPAW6Aw,rBzKTkUG3Wk,2024-08-31,13:00:57,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,19,274,0,40821,hd,1:01:09,False,Not Restricted,public,False
6,UC295-Dw_tDNtZXFeAPAW6Aw,XHQP6mut8YI,2024-09-01,11:00:36,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,27,485,0,82374,hd,0:20:16,False,Not Restricted,public,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45357,UCf1U16LSYc8QhortB6Bg1Zw,3_hSnZsG1i0,2024-07-18,13:54:19,Diy Craft and Art ideas 💫,UUf1U16LSYc8QhortB6Bg1Zw,12,11,1906,22,...,0,11,0,413,hd,0:00:32,False,Not Restricted,public,False
45358,UCf1U16LSYc8QhortB6Bg1Zw,Uev83QY04h8,2024-08-12,13:59:04,Diy Craft and Art ideas 💫,UUf1U16LSYc8QhortB6Bg1Zw,12,11,1906,22,...,0,4,0,12,hd,0:00:15,False,Not Restricted,public,False
45359,UCf1U16LSYc8QhortB6Bg1Zw,s4gMPgVB0N4,2024-08-12,04:18:37,Diy Craft and Art ideas 💫,UUf1U16LSYc8QhortB6Bg1Zw,12,11,1906,22,...,0,4,0,20,hd,0:00:08,False,Not Restricted,public,False
45371,UCco0YFnoXvvXL5tD6Cc7Lbw,MqZ-rzkT6Ck,2024-12-06,15:31:20,Eve diy and arts,UUco0YFnoXvvXL5tD6Cc7Lbw,5,2,215,22,...,0,7,0,98,hd,0:00:12,False,Not Restricted,public,False


In [16]:
csv_output_path = "data/english_video_data.csv"
cleaned_videos_df.to_csv(csv_output_path, index=False)